<a href="https://colab.research.google.com/github/saiPraneeth1608/saipraneeth_INFO5731_Spring2020/blob/main/INFO5731_Assignment_Four.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Four**

In this assignment, you are required to conduct topic modeling, sentiment analysis based on **the dataset you created from assignment three**.

# **Question 1: Topic Modeling**

(30 points). This question is designed to help you develop a feel for the way topic modeling works, the connection to the human meanings of documents. Based on the dataset from assignment three, write a python program to **identify the top 10 topics in the dataset**. Before answering this question, please review the materials in lesson 8, especially the code for LDA and LSA. The following information should be reported:

(1) Features (top n-gram phrases) used for topic modeling.

(2) Top 10 clusters for topic modeling.

(3) Summarize and describe the topic for each cluster. 

In [ ]:
import json
import zipfile
import glob
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
words = stopwords.words('english')
from textblob import Word
from textblob import TextBlob
nltk.download('punkt')
nltk.download('wordnet')
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Write your code here
!pip install tomotopy




In [ ]:
import spacy
import os
import logging
import nltk
import numpy as np
import matplotlib.pyplot as plt
import tomotopy as tp
import numpy as np
import pyLDAvis
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
!pip install pyLDAvis

In [ ]:
#data is loaded here
with zipfile.ZipFile('/content/sample_data/Assignment four data Yelp (question 1 and 2).zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Assi')

In [ ]:
datareviews=[]
for files in glob.iglob('/content/dataAssignment4/Assignment four data Yelp (question 1 and 2)/*'):
  f=open(files)
  data=json.load(f)
  for i in data:
    datareviews.append(i['text'])
print(" total reviews :",len(datareviews))

 total reviews : 500000


In [ ]:
dataframe = pd.DataFrame(reviews,columns=['review'])
dataframe.head(10)

,review
0,"Awesome service and fantastic food, the wait s..."
1,Seemed pretty pricey to me for what you got......
2,We TRIED to go to the buffet on Tuesday eve fo...
3,Came here for lunch on my 20th birthday and th...
4,"Amazing treats - I love their cinnamon roll, c..."
5,Me and my family went to TasteBuds tonight..I ...
6,Stay away from this train wreck of a hospital ...
7,I have now had two amazing meals here and can'...
8,My classmates and I go here because we go to c...
9,Really good service. They helped us pick out f...


In [ ]:
#here data is cleaned
#lower
dataframe['review']=dataframe['review'].map(lambda x: x.lower())
#removing punctutations
dataframe['review']=dataframe['review'].map(lambda x: re.sub('[?|!|\'|"|#]', '', x))
#Lemmitizing
dataframe['review']=dataframe['review'].apply(lambda x:" ".join(x for x in x.split() if x not in words))
dataframe['review']=dataframe['review'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
dataframe.head(10)

,review
0,"awesome service fantastic food, wait staff fri..."
1,seemed pretty pricey got.....burgers tasted pr...
2,"tried go buffet tuesday eve dinner, found clos..."
3,came lunch 20th birthday gave u free dessert s...
4,"amazing treat - love cinnamon roll, cupcakes, ..."
5,family went tastebud tonight..i love place beg...
6,"stay away train wreck hospital , er unprofessi..."
7,"two amazing meal cant wait return. steaks, lob..."
8,classmate go go cpcc walking distance class al...
9,really good service. helped u pick frame whole...


In [ ]:
#LDA algorithm
SLDA = tp.SLDAModel(k=20) 
for i in dataframe['review']:
    SLDA.add_doc(i.split())
for i in range(0, 100, 10):
    SLDA.train(10)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, SLDA.ll_per_word))
for k in range(SLDA.k):
    print('Top 15 words of topic #{}'.format(k))
    print(SLDA.get_topic_words(k, top_n=15))

SLDA.summary()

Iteration: 0	Log-likelihood: -10.289966547361333
Iteration: 10	Log-likelihood: -9.779854386387292
Iteration: 20	Log-likelihood: -9.529689170392261
Iteration: 30	Log-likelihood: -9.390326512890354
Iteration: 40	Log-likelihood: -9.315363755202982
Iteration: 50	Log-likelihood: -9.268753083975152
Iteration: 60	Log-likelihood: -9.238585417216376
Iteration: 70	Log-likelihood: -9.21779145654004
Iteration: 80	Log-likelihood: -9.202414399437433
Iteration: 90	Log-likelihood: -9.191106702906314
Top 15 words of topic #0
[('ice', 0.015714650973677635), ('cream', 0.014101624488830566), ('tea', 0.013097194023430347), ('coffee', 0.011876042932271957), ('chocolate', 0.010320412926375866), ('cake', 0.009061657823622227), ('like', 0.008468895219266415), ('sweet', 0.0071052443236112595), ('also', 0.006838055793195963), ('flavor', 0.006642117630690336), ('one', 0.006185918115079403), ('drink', 0.006182949524372816), ('place', 0.006093886680901051), ('try', 0.0058405520394444466), ('dessert', 0.005719822365

In [ ]:
topic_dists = np.stack([SLDA.get_topic_word_dist(k) for k in range(SLDA.k)])
doc_dists = np.stack([doc.get_topic_dist() for doc in SLDA.docs])
document_lengths = np.array([len(doc.words) for doc in SLDA.docs])
vocab = list(SLDA.used_vocabs)
term_frequency = SLDA.used_vocab_freq

pre_data = pyLDAvis.prepare(
    topic_dists, 
    doc_dists, 
    document_lengths, 
    vocab, 
    term_frequency
)
pyLDAvis.save_html(pre_data, 'ldavis.html')

In [ ]:
pyLDAvis.display(pre_data)

In [ ]:
!pip install hdbscan

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip install sentence_transformers

     |████████████████████████████████| 71kB 3.7MB/s 
     |████████████████████████████████| 1.3MB 8.8MB/s 
     |████████████████████████████████| 2.9MB 40.4MB/s 
     |████████████████████████████████| 1.1MB 45.7MB/s 
     |████████████████████████████████| 890kB 40.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.9-cp36-none-any.whl size=101036 sha256=a898f5f9c0f3be98e14255d526d3218039cd79f09056a901164065b8308ecc57
  Stored in directory: /root/.cache/pip/wheels/fc/89/43/f2f5bc00b03ef9724b0f6254a97eaf159a4c4ddc024b33e07a
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=91700e9487546a7516df6cf454db0e2e638c9421402c644225ca24e1cbbf6efa
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [ ]:


import umap
import hdbscan
from sentence_transformers import SentenceTransformer
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import matplotlib.pyplot as plt


def c_tf_idf(documents, m, ngram_range=(1, 1)):
    """ Calculate a class-based TF-IDF where m is the number of total documents. """
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count

def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topic'])
                     .Doc
                     .count()
                     .reset_index()
                     .rename({"Topic": "Topic", "Doc": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

In [ ]:
data = dataframe[:5000]

In [ ]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
 
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
docs_df = pd.DataFrame(data, columns=["Doc"])
docs_df['Topic'] = cluster.labels_
docs_df['Doc_ID'] = range(len(docs_df))
docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})

In [ ]:
tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m = len(data))

In [ ]:
top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)
topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)

In [ ]:
top_n_words

{-1: [('like', 0.007853674962756714),
  ('just', 0.007712689474090459),
  ('time', 0.0073492009389198995),
  ('good', 0.007342011050797255),
  ('great', 0.007332016100548271),
  ('service', 0.007191062625283349),
  ('place', 0.007163363217997229),
  ('really', 0.007092395353321187),
  ('food', 0.0070143095955854514),
  ('im', 0.006740559703143957),
  ('nice', 0.00673283695978155),
  ('ive', 0.006518978387713805),
  ('little', 0.006364147954385099),
  ('restaurant', 0.00635550231917936),
  ('order', 0.0062115498544323125),
  ('got', 0.006205057459537922),
  ('ordered', 0.006199898171568424),
  ('dont', 0.006118056806747686),
  ('try', 0.006100725310665647),
  ('came', 0.006098406869834275)],
 0: [('sushi', 0.12988246536199421),
  ('roll', 0.0522307787918433),
  ('rolls', 0.03086985676171022),
  ('salmon', 0.026325524575151325),
  ('japanese', 0.026020021027251886),
  ('tuna', 0.025227037033649567),
  ('nigiri', 0.024964032834888666),
  ('unagi', 0.021401928936339413),
  ('crab', 0.01812

# **Question 2: Yelp Review Sentiment Analysis**

(30 points). Sentiment analysis also known as opinion mining is a sub field within Natural Language Processing (NLP) that builds machine learning algorithms to classify a text according to the sentimental polarities of opinions it contains, e.g., positive, negative, neutral. The purpose of this question is to develop a machine learning classifier for sentiment analysis. Based on the dataset from assignment three, write a python program to implement a sentiment classifier and evaluate its performance. Notice: **80% data for training and 20% data for testing**.  

(1) Features used for sentiment classification and explain why you select these features.

(2) Select two of the supervised learning algorithm from scikit-learn library: https://scikit-learn.org/stable/supervised_learning.html#supervised-learning, to build a sentiment classifier respectively. 

(3) Compare the performance over accuracy, precision, recall, and F1 score for the two algorithms you selected. Here is the reference of how to calculate these metrics: https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9.

In [ ]:
"""Slant investigation, which is additionally called assessment mining, audit mining (evaluation extraction), or mentality examination, is the assignment of distinguishing, separating, and ordering feelings, notions, and perspectives concerning various themes. In an AI approach, estimation examination can be considered as a directed grouping task.We have distinctive component choice strategies, for example, Mutual Information,Information Gain,Chi-square,Tf-idf,lexicon, word vectors and some more. 

Shared Information: MIterm chooses highlights that are not consistently dispersed among the assessment classes since they are enlightening of their classes. 

Chi-square:Chi-square estimates what amount expected checks and noticed tallies go astray from one another. 

Data Gain: Information gainis the most regularly utilized component determination strategy in the field of AI. It computes the pertinence of a component for expectation of assumption of survey by breaking down the presence or nonattendance of an element in an archive. 

TF-Idf: It is a weighting plan, which quantifies how important a word to an example in the dataset.The pertinence increments when the occasions a word shows up in the example 

Other component extraction procedure is word embeddings (otherwise called word vectors). 

Here for this investigation, I have utilized TF-Idf, which is a factual measure that assesses how pertinent a word is to an archive in an assortment of reports. This is finished by increasing two measurements: how often a word shows up in a report, and the reverse record recurrence of the word over a bunch of documents.Machine learning with regular language is confronted with one significant obstacle – its calculations generally manage numbers, and common language is, well, text. So we need to change that text into numbers(text vectorization). It's a principal step during the time spent AI for investigating information, and diverse vectorization calculations will definitely influence outcome, so we need to pick one that will convey the outcomes we are seeking after. 

Whenever we have changed words into numbers, such that's AI calculations can comprehend, the TF-IDF score can be taken care of to calculations, for example, Naive Bayes and Support Vector Machines and so on, enormously improving the consequences of more fundamental strategies like word tallies. 

So this sort of portrayals makes it workable for words with comparable importance to have a comparable portrayal, which can improve the presentation of classifiers."""

In [ ]:
# Write your code here
import pandas as pd
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')
import json
import zipfile
import glob
import json
import zipfile
import glob
from textblob import TextBlob





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
with zipfile.ZipFile('/content/sample_data/Assignment four data Yelp (question 1 and 2).zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Assi')
myrate = [ ]
mytext = [ ]
for files in glob.iglob('/content/Assi/Assignment four data Yelp (question 1 and 2)/*'):
  f=open(files)
  data=json.load(f)
  for i in data:
      myrate.append(i['stars'])
      mytext.append(i['text'])

In [ ]:
dd = pd.DataFrame({'Reviews':mytext,'StaRat':myrate})


In [ ]:
dd['StaRat']=dd['StaRat'].astype(int)

In [ ]:
#data is processed here
dd['cleanedtext'] = dd['Reviews'].apply(lambda x: " ".join(x.lower() for x in x.split()))
dd['cleanedtext'] = dd['cleanedtext'].astype(str).str.replace('[^\w\s]','')
dd['cleanedtext'] = dd['cleanedtext'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
high_freq = pd.Series(' '.join(dd['cleanedtext']).split()).value_counts()[:10]
high_freq = list(high_freq.index)
dd['cleanedtext'] = dd['cleanedtext'].apply(lambda x: " ".join(x for x in x.split() if x not in high_freq))
low_freq = pd.Series(' '.join(dd['cleanedtext']).split()).value_counts()[-10:]
low_freq = list(low_freq.index)
dd['cleanedtext'] = dd['cleanedtext'].apply(lambda x: " ".join(x for x in x.split() if x not in low_freq))

dd['cleanedtext'] = dd['Reviews'].astype(str).str.replace('[^\w\s]','')
dd['cleanedtext'] = dd['cleanedtext'].apply(lambda x: " ".join(x.lower() for x in x.split()))
dd['cleanedtext'] = dd['cleanedtext'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score,  precision_score, recall_score,accuracy_score
def sentiment(x):
    if x == 5:
        return 'Very positive'
    if x == 4:
        return 'Positive'
    if x == 3:
        return 'Neutral'
    if x == 2:
        return 'Negative'
    if x == 1:
        return 'Very negative'
dd['sentiment'] = dd['StaRat'].apply(lambda x: sentiment(x))
train, test = sklearn.model_selection.train_test_split(dd, train_size=0.8, test_size=0.2)

In [ ]:
#Linear Support Vector Classifier
support_vector = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=100, 
                                               learning_rate='optimal', tol=None))])
svm = support_vector.fit(train['cleanedtext'], train['sentiment'])
test['predicted sentiment'] = svm.predict(test['cleanedtext'])
print(test[['Reviews', 'sentiment', 'predicted sentiment']].head(5))
svm_recall = recall_score(test['sentiment'], test['predicted sentiment'], average='macro')
svm_precision = precision_score(test['sentiment'], test['predicted sentiment'], average='macro')
svm_accuracy = accuracy_score(test['sentiment'], test['predicted sentiment'])*100
svm_f1 = f1_score(test['sentiment'], test['predicted sentiment'], average='macro')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


                                                  Reviews  ... predicted sentiment
146008  Tried the new location on rural and broadway. ...  ...       Very positive
345982  If I could give zero stars I would. Wait staff...  ...       Very negative
372068  Damn good breakfast ... the pancakes in partic...  ...       Very positive
291121  Aesthetically this place is very good but the ...  ...            Positive
206430  Came here with my friends during the holidays ...  ...       Very positive

[5 rows x 3 columns]


In [ ]:
print('Accuracy  :', svm_accuracy)
print('f1-score :', svm_f1)
print('Precision :', svm_precision)
print('Recall score :', svm_recall)

Accuracy  : 56.718999999999994
f1-score : 0.35275832992477185
Precision : 0.5269920727602359
Recall score : 0.379472324991674


In [ ]:
nb_pipeline = Pipeline([('vect', CountVectorizer()),
                        ('tfidf', TfidfTransformer()),
                        ('clf', MultinomialNB())])
nb = nb_pipeline.fit(train['cleanedtext'], train['sentiment'])
test['nb predicted sentiment'] = nb.predict(test['cleanedtext'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
print('\n', 'MultinomialNB SENTIMENT ANALYSIS:', '\n')
print(test[['Reviews', 'sentiment', 'nb predicted sentiment']].head(10))
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
nb_recall = recall_score(test['sentiment'], test['nb predicted sentiment'], average='macro')
nb_precision = precision_score(test['sentiment'], test['nb predicted sentiment'], average='macro')
nb_accuracy = accuracy_score(test['sentiment'], test['nb predicted sentiment'])*100
nb_f1 = f1_score(test['sentiment'], test['nb predicted sentiment'], average='macro')

print('Accuracy of MultinomialNB:', nb_accuracy)
print('f1-score of MultinomialNB:', nb_f1)
print('Precision score of MultinomialNB:', nb_precision)
print('Recall score of  MultinomialNB:', nb_recall)


 MultinomialNB SENTIMENT ANALYSIS: 

                                                  Reviews  ... nb predicted sentiment
146008  Tried the new location on rural and broadway. ...  ...          Very positive
345982  If I could give zero stars I would. Wait staff...  ...          Very negative
372068  Damn good breakfast ... the pancakes in partic...  ...          Very positive
291121  Aesthetically this place is very good but the ...  ...          Very positive
206430  Came here with my friends during the holidays ...  ...          Very positive
4146    Amazing pizza! Good pasta too. I had delivery ...  ...          Very positive
396986  $5.50 for a large soda & NO REFILL!!!!! I get ...  ...          Very positive
238975  Awesome find!! From the decor, to the menu, to...  ...          Very positive
415652  Trying hard to be a LA street food truck. \n\n...  ...          Very positive
239476  I came across this clinic while looking for a ...  ...          Very positive

[10 rows x 3 co

# **Question 3: House price prediction**

(40 points). You are required to build a **regression** model to predict the house price with 79 explanatory variables describing (almost) every aspect of residential homes. The purpose of this question is to practice regression analysis, an supervised learning model. The training data, testing data, and data description files can be download here: https://github.com/unt-iialab/info5731_spring2021/blob/main/assignment/assignment4-question3-data.zip. Here is an axample for implementation: https://towardsdatascience.com/linear-regression-in-python-predict-the-bay-areas-home-price-5c91c8378878. 


In [ ]:
# Write your code h
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import nltk
import math
import time
import re
import matplotlib.pyplot as plt


In [ ]:

mytrain_data=pd.read_csv("/content/sample_data/train.csv")
mytest_data=pd.read_csv("/content/sample_data/test.csv")


In [ ]:
print(mytrain_data.isnull().sum())
mytrain_data = mytrain_data.replace(np.NaN,0)
print(mytrain_data.isnull().sum())

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64
Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 81, dtype: int64


In [ ]:
print(mytest_data.isnull().sum())
mytest_data = mytest_data.replace(np.NaN,0)
print(mytest_data.isnull().sum())

Id                 0
MSSubClass         0
MSZoning           4
LotFrontage      227
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           1
SaleCondition      0
Length: 80, dtype: int64
Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
Length: 80, dtype: int64


In [ ]:
column = list(mytrain_data.select_dtypes(['object']).columns)

In [ ]:
mytrain_data[column] = mytrain_data[column].apply(lambda x: x.astype('category'))
mytrain_data[column] = mytrain_data[column].apply(lambda x: x.cat.codes)

In [ ]:
mytest_data[column] = mytest_data[column].apply(lambda x: x.astype('category'))
mytest_data[column] = mytest_data[column].apply(lambda x: x.cat.codes)

In [ ]:
feat_col = ['OverallQual','YearBuilt','YearRemodAdd','MasVnrArea','ExterQual','BsmtFinSF1','TotalBsmtSF','1stFlrSF','GrLivArea','FullBath','KitchenQual','TotRmsAbvGrd','Fireplaces','FireplaceQu','GarageCars','GarageArea']

In [ ]:
# Linear Model of train and test
X = mytrain_data[feat_col]
Y = mytrain_data['SalePrice']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)


In [ ]:
mytest =mytest_data[feat_col]

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
predicted = regressor.predict(mytest)

In [ ]:
print(pd.DataFrame({'Predicted Price':predicted}))

      Predicted Price
0        91999.486628
1       146014.569048
2       152275.661837
3       181545.347071
4       196500.106334
...               ...
1454     57780.813533
1455     79833.635129
1456    150966.348431
1457     93242.203392
1458    216229.480285

[1459 rows x 1 columns]
